## Historical people: Quick and dirty

This example shows how to get some initial record linkage results as quickly as possible.

There are many ways to improve the accuracy of this model. But this may be a good place to start if you just want to give Splink a try and see what it's capable of.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/duckdb/quick_and_dirty_persons.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev

In [2]:
from splink.datasets import splink_datasets

df = splink_datasets.historical_50k
df.head(5)

,unique_id,cluster,full_name,first_and_surname,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
2,Q2296770-3,Q2296770,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician
3,Q2296770-4,Q2296770,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,None,politician
4,Q2296770-5,Q2296770,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,None,politician


In [3]:
from splink import block_on, SettingsCreator
import splink.comparison_library as cl
import splink.comparison_template_library as ctl


settings = SettingsCreator(
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=[
        block_on("full_name"),
        block_on("substr(full_name,1,6)", "dob", "birth_place"),
        block_on("dob", "birth_place"),
        block_on("postcode_fake"),
    ],
    comparisons=[
        cl.JaroWinklerAtThresholds("full_name", [0.9, 0.7]).configure(
            term_frequency_adjustments=True
        ),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["day", "month", "year"],
            datetime_thresholds=[5, 1, 5],
        ),
        cl.LevenshteinAtThresholds("postcode_fake", 2),
        cl.JaroWinklerAtThresholds("birth_place", 0.9).configure(
            term_frequency_adjustments=True
        ),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ],
)

In [4]:
from splink import Linker, DuckDBAPI


linker = Linker(df, settings, database_api=DuckDBAPI(), set_up_basic_logging=False)
deterministic_rules = [
    "l.full_name = r.full_name",
    "l.postcode_fake = r.postcode_fake and l.dob = r.dob",
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

In [5]:
linker.training.estimate_u_using_random_sampling(max_pairs=2e6)

In [6]:
results = linker.inference.predict(threshold_match_probability=0.9)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'full_name':
    m values not fully trained
Comparison: 'dob':
    m values not fully trained
Comparison: 'postcode_fake':
    m values not fully trained
Comparison: 'birth_place':
    m values not fully trained
Comparison: 'occupation':
    m values not fully trained


In [7]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,full_name_l,full_name_r,gamma_full_name,dob_l,dob_r,gamma_dob,postcode_fake_l,postcode_fake_r,gamma_postcode_fake,birth_place_l,birth_place_r,gamma_birth_place,occupation_l,occupation_r,gamma_occupation,match_key
0,3.172246,0.900145,Q16198727-6,Q16198727-8,henry jupp,jupp,0,1802-08-06,1802-08-06,5,None,e4 9re,-1,waltham forest,waltham forest,2,cricketer,cricketer,1,2
1,3.172423,0.900156,Q16220644-12,Q16220644-7,1st bt.,1st bt.,3,1840-11-21,1810-11-21,4,None,None,-1,liverpool,liverpool,2,None,physician,-1,0
2,3.173256,0.900208,Q6180874-12,Q6180874-19,richard slater,slater,0,1854-01-01,1854-01-01,5,al5 2ay,al3 7rq,0,st albans,st albans,2,hymnwriter,hymnwriter,1,2
3,3.174182,0.900265,Q7519167-10,Q7519167-8,simeon langton,simeon langton,3,1150-01-81,1152-01-01,0,None,None,-1,wealden,wealden,2,priest,None,-1,0
4,3.178567,0.900538,Q15980561-12,Q15980561-8,harry rosling,henry rosling,1,1828-01-11,1858-01-01,0,tn27 0sy,tn27 0sy,2,ashford,ashford,2,photographer,None,-1,3
